### Estimate Partition Count for File Read

In [57]:
# Create Spark Session

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Estimate Partition Count") \
    .master("local[*]") \
    .getOrCreate()

spark

In [58]:
# Check the default partition size
partition_size = int(spark.conf.get("spark.sql.files.maxPartitionBytes").replace("b",""))
print(f"Partition Size: {partition_size} in bytes and {int(partition_size) / 1024 / 1024} in MB")

# Check the default open Cost in Bytes
open_cost_size = int(spark.conf.get("spark.sql.files.openCostInBytes").replace("b",""))
print(f"Open Cost Size: {open_cost_size} in bytes and {int(open_cost_size) / 1024 / 1024} in MB")

# Default parallelism
parallelism = int(spark.sparkContext.defaultParallelism)
print(f"Default Parallelism: {parallelism}")

Partition Size: 134217728 in bytes and 128.0 in MB
Open Cost Size: 4194304 in bytes and 4.0 in MB
Default Parallelism: 8


In [59]:
# File size in Bytes
average_file_size = 2647733632
total_files = 1

# Total Actual File Size in Bytes
total_file_size = average_file_size * total_files
print(f"Total File size on disk: {total_file_size} in bytes and {total_file_size / 1024 /1024} in MB")

Total File size on disk: 2647733632 in bytes and 2525.0755615234375 in MB


In [60]:
# Padded file size for Spark read
padded_file_size = total_file_size + (total_files * open_cost_size)
print(f"Total padded file size: {padded_file_size} in bytes and {padded_file_size / 1024 /1024} in MB")

Total padded file size: 2651927936 in bytes and 2529.0755615234375 in MB


In [61]:
# Number of Bytes per Core
bytes_per_core = padded_file_size / parallelism
print(f"Bytes per Core: {bytes_per_core} in bytes and {bytes_per_core / 1024 /1024} in MB")

Bytes per Core: 331490992.0 in bytes and 316.1344451904297 in MB


In [62]:
# Max Split Bytes
max_bytes_per_split = min(partition_size, max(open_cost_size, bytes_per_core))
print(f"Max bytes per Partition: {max_bytes_per_split} in bytes and {max_bytes_per_split / 1024 /1024} in MB")

Max bytes per Partition: 134217728 in bytes and 128.0 in MB


In [63]:
# Total number of Partitions
num_of_partitions = padded_file_size / max_bytes_per_split
print(f"Approx number of partitions: {num_of_partitions}")

Approx number of partitions: 19.758402824401855


In [64]:
# Read the file to see the number of partitons
df_1 = spark.read.format("csv").option("header", True).load("dataset/sales_combined_2.csv")
print(f"Number of Partition -> {df_1.rdd.getNumPartitions()}")

Number of Partition -> 20


In [51]:
# Lets pack all code in single function
def num_partitions(file_size, num_of_files, spark):
    # Check the default partition size
    partition_size = int(spark.conf.get("spark.sql.files.maxPartitionBytes").replace("b",""))
    # Check the default open Cost in Bytes
    open_cost_size = int(spark.conf.get("spark.sql.files.openCostInBytes").replace("b",""))
    # Default parallelism
    parallelism = int(spark.sparkContext.defaultParallelism)
    # Total Actual File Size in Bytes
    total_file_size = file_size * num_of_files
    # Padded file size for Spark read
    padded_file_size = total_file_size + (num_of_files * open_cost_size)
    # Number of Bytes per Core
    bytes_per_core = padded_file_size / parallelism
    # Max Split Bytes
    max_bytes_per_split = min(partition_size, max(open_cost_size, bytes_per_core))
    # Total number of Partitions
    num_of_partitions = padded_file_size / max_bytes_per_split
    
    return num_of_partitions

In [52]:
# Validation 1
# Calculate the number of partitions as per our logic
estimated_num_partition = num_partitions(2647733632, 1, spark)
print(f"Estimated number of partitions = {estimated_num_partition}")

# Lets read a single csv file with approx size 2647733632 bytes or 2.5 GB
df_1 = spark.read.format("csv").option("header", True).load("dataset/sales_combined_2.csv")
print(f"Number of Partition -> {df_1.rdd.getNumPartitions()}")


Estimated number of partitions = 19.758402824401855
Number of Partition -> 20


In [53]:
# Validation 2
# Calculate the number of partitions as per our logic for tiny files
estimated_num_partition = num_partitions(7777, 41300, spark)
print(f"Estimated number of partitions = {estimated_num_partition}")

# Lets read multiple parquet file with approx size 7777 bytes or 7.7 KB
df_2 = spark.read.format("parquet").load("dataset/sales_trx_id.parquet/")
print(f"Number of Partition -> {df_2.rdd.getNumPartitions()}")

Estimated number of partitions = 1293.018052726984
Number of Partition -> 1292


In [54]:
# Validation 3
# Calculate the number of partitions as per our logic for avg files
estimated_num_partition = num_partitions(1159176, 220, spark)
print(f"Estimated number of partitions = {estimated_num_partition}")

# Lets read multiple parquet file with approx size 1159176 bytes or 1.1 MB
df_3 = spark.read.format("parquet").load("dataset/sales_city_id.parquet/")
print(f"Number of Partition -> {df_3.rdd.getNumPartitions()}")

Estimated number of partitions = 8.77503752708435
Number of Partition -> 8


In [55]:
spark.stop()